In [265]:
# autoreload
%load_ext autoreload
%autoreload 2

# fix path
import sys
sys.path.append("/home/jxm/reverse-training")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [266]:
from utils.analysis import load_results_as_df
import os


HOME_DIR = "/home/jxm/reverse-training"
df = load_results_as_df(os.path.join(HOME_DIR, "results"))
print("Length:", len(df))


Filtered 0/284 results with no evaluation metrics
dict_keys(['data', 'full_ot_distance', 'sinkhorn_distance', 'jaccard_overlap_examples', 'jaccard_overlap_vocabulary', 'levenshtein_stats', 'args', 'expert_evaluation_metrics', 'evaluation_metrics'])
Length: 284


In [267]:
df["dataset"].value_counts()

dataset
ag_news_10000      96
dbpedia_10000      95
newsgroup_10000    93
Name: count, dtype: int64

In [268]:
[col for col in df.columns if "acc" in col]

['expert_evaluation_best_eval_accuracy',
 'evaluation_best_eval_accuracy',
 'dataset_jaccard_overlap_examples',
 'dataset_jaccard_overlap_vocabulary']

In [269]:
import pandas as pd

pd.set_option('display.max_rows', None)

dv_cols = ["dataset", "select_seed_dataset", "select_full_dataset_size", "select_batch_fill_strategy"]

iv_cols = ["evaluation_best_eval_accuracy", "dataset_sinkhorn_distance", "dataset_jaccard_overlap_vocabulary"]

# final_df = df[df["select_seed_dataset"] == "nq_500000"]

df.groupby(dv_cols)[iv_cols].mean().sort_values(
    by=["dataset","select_seed_dataset", "select_full_dataset_size", "select_batch_fill_strategy"], 
    ascending=False
)

evaluation_best_eval_accuracy  \
dataset         select_seed_dataset select_full_dataset_size select_batch_fill_strategy                                  
newsgroup_10000 nq_100000           2500                     topk                                             0.069010   
                                                             random                                           0.144694   
                                                             bottomk                                          0.068034   
                                    1000                     topk                                             0.062826   
                                                             random                                           0.106445   
                                                             greedy_batched                                   0.119466   
                                                             bottomk                                          0.064128   
                                    500                      topk                                             0.054036   
                                                             random                                           0.082520   
                                                             greedy_batched                                   0.088867   
                                                             bottomk                                          0.051432   
                                    100                      topk                                             0.032878   
                                                             random                                           0.047689   
                                                             greedy_batched                                   0.032064   
                                                             bottomk                                          0.035645   
                nq_10000            2500                     topk                                             0.099447   
                                                             random                                           0.147949   
                                                             greedy_batched                                   0.174154   
                                                             bottomk                                          0.098633   
                                    1000                     topk                                             0.068359   
                                                             random                                           0.107585   
                                                             greedy_batched                                   0.133952   
                                                             bottomk                                          0.070312   
                                    500                      topk                                             0.049805   
                                                             random                                           0.088867   
                                                             greedy_batched                                   0.107910   
                                                             bottomk                                          0.053548   
                                    100                      topk                                             0.032552   
                                                             random                                           0.029297   
                                                             greedy_batched                                   0.050618   
                                                             bottomk                                          0.034424   
dbpedia_10000   nq_100000           2500                     to

In [270]:


dv_cols = ["dataset", "select_seed_dataset", "select_full_dataset_size", "select_batch_fill_strategy"]

iv_cols = ["evaluation_best_eval_accuracy", "dataset_sinkhorn_distance", "dataset_levenshtein_stats_average_distance", "dataset_full_ot_distance"]


# Final results: dataset size 1000, batch fill strategy is not bottomk
final_df = df[
    (df["select_full_dataset_size"] == 1000) & 
    (df["select_batch_fill_strategy"] != "bottomk") &
    (df["dataset"] != "newsgroup_10000")
].reset_index()
final_df = final_df.groupby(dv_cols)[iv_cols].mean().sort_values(
    by=["dataset","select_seed_dataset", "select_full_dataset_size", "select_batch_fill_strategy"], 
    ascending=False
)
final_df = final_df.reset_index()
final_df = final_df.pivot(index=["select_seed_dataset", "select_batch_fill_strategy"], columns=["dataset"], values=iv_cols)
final_df = final_df.swaplevel(0, 1, axis=1)
final_df = final_df.sort_index(axis=1)
final_df

dataset                                                   ag_news_10000  \
                                               dataset_full_ot_distance   
select_seed_dataset select_batch_fill_strategy                            
nq_10000            greedy_batched                             1.031292   
                    random                                     1.055060   
                    topk                                       1.052283   
nq_100000           greedy_batched                             1.043698   
                    random                                     1.052631   
                    topk                                       1.050810   

dataset                                                                                    \
                                               dataset_levenshtein_stats_average_distance   
select_seed_dataset select_batch_fill_strategy                                              
nq_10000            greedy_batched                                             168.459733   
                    random                                                     168.475533   
                    topk                                                       168.629933   
nq_100000           greedy_batched                                             168.560367   
                    random                                                     168.626267   
                    topk                                                       168.761233   

dataset                                                                   \
                                               dataset_sinkhorn_distance   
select_seed_dataset select_batch_fill_strategy                             
nq_10000            greedy_batched                              1.273789   
                    random                                      1.292871   
                    topk                                        1.288595   
nq_100000           greedy_batched                              1.283533   
                    random                                      1.291164   
                    topk                                        1.286754   

dataset                                                                       \
                                               evaluation_best_eval_accuracy   
select_seed_dataset select_batch_fill_strategy                                 
nq_10000            greedy_batched                                  0.753255   
                    random                                          0.668294   
                    topk                                            0.427246   
nq_100000           greedy_batched                                  0.677246   
                    random                                          0.670410   
                    topk                                            0.380046   

dataset                                                   dbpedia_10000  \
                                               dataset_full_ot_distance   
select_seed_dataset select_batch_fill_strategy                            
nq_10000            greedy_batched                             0.926802   
                    random                                     0.966919   
                    topk                                       1.027396   
nq_100000           greedy_batched                             0.934150   
                    random                                     0.965468   
                    topk                                       1.017106   

dataset                                                                                    \
                                               dataset_levenshtein_stats_average_distance   
select_seed_dataset select_batch_fill_strategy                                              
nq_10000            greedy_batched                                             220.947700   
                    random      

In [263]:
print(final_df.to_latex(float_format=lambda x: f"{100*x:.1f}", escape=True))

\begin{tabular}{llrrrrrrrr}
\toprule
 & dataset & \multicolumn{4}{r}{ag\_news\_10000} & \multicolumn{4}{r}{dbpedia\_10000} \\
 &  & dataset\_full\_ot\_distance & dataset\_levenshtein\_stats\_average\_distance & dataset\_sinkhorn\_distance & evaluation\_best\_eval\_accuracy & dataset\_full\_ot\_distance & dataset\_levenshtein\_stats\_average\_distance & dataset\_sinkhorn\_distance & evaluation\_best\_eval\_accuracy \\
select_seed_dataset & select_batch_fill_strategy &  &  &  &  &  &  &  &  \\
\midrule
\multirow[t]{3}{*}{nq\_10000} & greedy\_batched & 103.1 & 16846.0 & 127.4 & 75.3 & 92.7 & 22094.8 & 116.9 & 63.9 \\
 & random & 105.5 & 16847.6 & 129.3 & 66.8 & 96.7 & 22134.6 & 120.6 & 55.0 \\
 & topk & 105.2 & 16863.0 & 128.9 & 42.7 & 102.7 & 22231.7 & 124.7 & 20.2 \\
\cline{1-10}
\multirow[t]{3}{*}{nq\_100000} & greedy\_batched & 104.4 & 16856.0 & 128.4 & 67.7 & 93.4 & 22113.2 & 117.7 & 50.0 \\
 & random & 105.3 & 16862.6 & 129.1 & 67.0 & 96.5 & 22098.6 & 120.4 & 41.7 \\
 & topk & 105.1

In [264]:
df.columns

Index(['dataset', 'dataset_size', 'minibatch_size', 'eval_every',
       'max_iterations', 'max_iterations_x', 'pretrain_iterations_x',
       'lr_tokens', 'lr_lr', 'lr_teacher', 'syn_steps', 'seed', 'token_init',
       'expert_epochs', 'sequence_length', 'num_experts',
       'num_steps_per_expert', 'expert_batch_size', 'expert_lr',
       'discrete_optimizer', 'admm_penalty_term', 'gcg_search_width',
       'gcg_tokens_to_swap', 'gcg_documents_to_swap', 'select_seed_dataset',
       'select_projection_dim', 'select_steps_per_grad',
       'select_full_dataset_size', 'select_lr_student',
       'select_num_pseudoexperts', 'select_do_classification',
       'select_batch_fill_strategy', 'select_label_strategy',
       'select_do_warmup', 'select_grads_full_model', 'num_eval_epochs',
       'exp_name', '_wandb', 'expert_evaluation_best_eval_loss',
       'expert_evaluation_best_eval_accuracy', 'evaluation_best_eval_loss',
       'evaluation_best_eval_accuracy', 'dataset_sinkhorn_distan